# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The RNN decoder is based on the architecture described in the paper (https://arxiv.org/pdf/1411.4555.pdf). There are suggested values for hyperparameters (vocab_threshold=5, embed_size=hidden_size=512) that I used in building my RNN. The CNN encoder was provided and is a pretrained ResNet model. I tried  different batch sizes and settled on 128 which allowed for faster modeling without affecting loss too much.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.


### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The provided transform_train function provided only flips the image to augment so I added transforms.RandomRotation(degrees=(-20,20)) function for extra augementation.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** The CNN encoder is pretrained so most of the parameters did not require training. I only had to modify the last connected layer to conect the CNN output to the RNN. ResNet is a good choice of CNN as it has been pretrained on a large image dataset so will have already been calibrated for the best results. The RNN decoder requires training from scratch.  

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** The above paper suggests using the SGD optimiser but after training for a few hours, it was very slow. I decided to change to the Adam optimiser using default parameters to speed up the model without affecting accuracy.

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import nltk
nltk.download('punkt')


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.RandomRotation(degrees=(-20,20)),     # rotate images +/- 20 degrees
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters())+ list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 370/414113 [00:00<01:51, 3698.65it/s]

  0%|          | 780/414113 [00:00<01:48, 3809.51it/s]

Done (t=0.98s)
creating index...
index created!
Obtaining caption lengths...




  0%|          | 1185/414113 [00:00<01:46, 3875.99it/s]

  0%|          | 1616/414113 [00:00<01:43, 3995.17it/s]

  0%|          | 2034/414113 [00:00<01:41, 4047.07it/s]

  1%|          | 2485/414113 [00:00<01:38, 4174.74it/s]

  1%|          | 2937/414113 [00:00<01:36, 4272.28it/s]

  1%|          | 3384/414113 [00:00<01:34, 4329.41it/s]

  1%|          | 3854/414113 [00:00<01:32, 4433.27it/s]

  1%|          | 4325/414113 [00:01<01:30, 4511.58it/s]

  1%|          | 4788/414113 [00:01<01:30, 4543.27it/s]

  1%|▏         | 5237/414113 [00:01<01:31, 4454.83it/s]

  1%|▏         | 5679/414113 [00:01<01:35, 4297.69it/s]

  1%|▏         | 6108/414113 [00:01<01:38, 4152.35it/s]

  2%|▏         | 6526/414113 [00:01<01:37, 4159.18it/s]

  2%|▏         | 6973/414113 [00:01<01:35, 4245.64it/s]

  2%|▏         | 7421/414113 [00:01<01:34, 4311.59it/s]

  2%|▏         | 7887/414113 [00:01<01:32, 4408.38it/s]

  2%|▏         | 8329/414113 [00:01<01:32, 4408.73it/s]

  2%|▏         | 8783/414113 

 14%|█▍        | 59361/414113 [00:13<01:23, 4241.34it/s]

 14%|█▍        | 59835/414113 [00:13<01:20, 4378.27it/s]

 15%|█▍        | 60293/414113 [00:13<01:20, 4382.45it/s]

 15%|█▍        | 60745/414113 [00:13<01:22, 4297.35it/s]

 15%|█▍        | 61214/414113 [00:13<01:20, 4407.85it/s]

 15%|█▍        | 61681/414113 [00:14<01:18, 4481.35it/s]

 15%|█▌        | 62135/414113 [00:14<01:18, 4477.93it/s]

 15%|█▌        | 62607/414113 [00:14<01:17, 4546.21it/s]

 15%|█▌        | 63074/414113 [00:14<01:16, 4580.13it/s]

 15%|█▌        | 63535/414113 [00:14<01:16, 4558.38it/s]

 15%|█▌        | 63997/414113 [00:14<01:16, 4575.97it/s]

 16%|█▌        | 64456/414113 [00:14<01:16, 4563.93it/s]

 16%|█▌        | 64914/414113 [00:14<01:17, 4531.39it/s]

 16%|█▌        | 65368/414113 [00:14<01:16, 4530.27it/s]

 16%|█▌        | 65822/414113 [00:15<01:18, 4447.00it/s]

 16%|█▌        | 66268/414113 [00:15<01:18, 4433.22it/s]

 16%|█▌        | 66714/414113 [00:15<01:18, 4440.43it/s]

 16%|█▌       

...

 43%|████▎     | 176635/414113 [00:39<00:59, 3985.26it/s]

 43%|████▎     | 177085/414113 [00:39<00:57, 4125.37it/s]

 43%|████▎     | 177527/414113 [00:39<00:56, 4208.70it/s]

 43%|████▎     | 177986/414113 [00:39<00:54, 4314.80it/s]

 43%|████▎     | 178429/414113 [00:39<00:54, 4312.33it/s]

 43%|████▎     | 178888/414113 [00:40<00:53, 4389.57it/s]

 43%|████▎     | 179337/414113 [00:40<00:53, 4418.53it/s]

 43%|████▎     | 179783/414113 [00:40<00:53, 4418.72it/s]

 44%|████▎     | 180238/414113 [00:40<00:52, 4456.71it/s]

 44%|████▎     | 180686/414113 [00:40<00:52, 4460.41it/s]

 44%|████▎     | 181134/414113 [00:40<00:54, 4301.79it/s]

 44%|████▍     | 181567/414113 [00:40<00:55, 4209.48it/s]

 44%|████▍     | 182013/414113 [00:40<00:54, 4280.43it/s]

 44%|████▍     | 182472/414113 [00:40<00:53, 4368.71it/s]

 44%|████▍     | 182920/414113 [00:41<00:52, 4399.32it/s]

 44%|████▍     | 183381/414113 [00:41<00:51, 4459.56it/s]

 44%|████▍     | 183842/414113 [00:41<00:51, 4502.84it/s

 57%|█████▋    | 234756/414113 [00:52<00:40, 4466.92it/s]

 57%|█████▋    | 235223/414113 [00:52<00:39, 4523.88it/s]

 57%|█████▋    | 235689/414113 [00:52<00:39, 4562.99it/s]

 57%|█████▋    | 236146/414113 [00:52<00:39, 4556.06it/s]

 57%|█████▋    | 236607/414113 [00:52<00:38, 4570.26it/s]

 57%|█████▋    | 237065/414113 [00:52<00:39, 4458.77it/s]

 57%|█████▋    | 237518/414113 [00:53<00:39, 4478.64it/s]

 57%|█████▋    | 237967/414113 [00:53<00:40, 4393.31it/s]

 58%|█████▊    | 238424/414113 [00:53<00:39, 4443.61it/s]

 58%|█████▊    | 238890/414113 [00:53<00:38, 4505.11it/s]

 58%|█████▊    | 239361/414113 [00:53<00:38, 4563.67it/s]

 58%|█████▊    | 239820/414113 [00:53<00:38, 4570.24it/s]

 58%|█████▊    | 240279/414113 [00:53<00:38, 4572.72it/s]

 58%|█████▊    | 240740/414113 [00:53<00:37, 4582.32it/s]

 58%|█████▊    | 241209/414113 [00:53<00:37, 4613.78it/s]

 58%|█████▊    | 241671/414113 [00:53<00:38, 4492.69it/s]

 58%|█████▊    | 242123/414113 [00:54<00:38, 4499.41it/s

 71%|███████   | 293912/414113 [01:05<00:25, 4805.65it/s]

 71%|███████   | 294396/414113 [01:05<00:24, 4814.73it/s]

 71%|███████   | 294878/414113 [01:05<00:24, 4791.40it/s]

 71%|███████▏  | 295358/414113 [01:05<00:25, 4713.70it/s]

 71%|███████▏  | 295830/414113 [01:05<00:25, 4639.02it/s]

 72%|███████▏  | 296295/414113 [01:05<00:25, 4628.18it/s]

 72%|███████▏  | 296771/414113 [01:05<00:25, 4665.91it/s]

 72%|███████▏  | 297238/414113 [01:05<00:25, 4631.47it/s]

 72%|███████▏  | 297703/414113 [01:06<00:25, 4634.85it/s]

 72%|███████▏  | 298183/414113 [01:06<00:24, 4681.53it/s]

 72%|███████▏  | 298652/414113 [01:06<00:24, 4681.24it/s]

 72%|███████▏  | 299121/414113 [01:06<00:26, 4367.39it/s]

 72%|███████▏  | 299563/414113 [01:06<00:26, 4362.04it/s]

 72%|███████▏  | 300027/414113 [01:06<00:25, 4440.74it/s]

 73%|███████▎  | 300474/414113 [01:06<00:25, 4381.88it/s]

 73%|███████▎  | 300949/414113 [01:06<00:25, 4483.60it/s]

 73%|███████▎  | 301442/414113 [01:06<00:24, 4607.42it/s

 85%|████████▌ | 352234/414113 [01:18<00:13, 4528.84it/s]

 85%|████████▌ | 352692/414113 [01:18<00:13, 4541.17it/s]

 85%|████████▌ | 353147/414113 [01:18<00:13, 4526.81it/s]

 85%|████████▌ | 353600/414113 [01:18<00:13, 4511.69it/s]

 85%|████████▌ | 354052/414113 [01:18<00:13, 4500.87it/s]

 86%|████████▌ | 354503/414113 [01:18<00:13, 4476.61it/s]

 86%|████████▌ | 354962/414113 [01:18<00:13, 4509.30it/s]

 86%|████████▌ | 355435/414113 [01:19<00:12, 4571.23it/s]

 86%|████████▌ | 355916/414113 [01:19<00:12, 4639.71it/s]

 86%|████████▌ | 356386/414113 [01:19<00:12, 4655.04it/s]

 86%|████████▌ | 356852/414113 [01:19<00:12, 4607.95it/s]

 86%|████████▋ | 357314/414113 [01:19<00:12, 4608.21it/s]

 86%|████████▋ | 357780/414113 [01:19<00:12, 4622.40it/s]

 87%|████████▋ | 358243/414113 [01:19<00:12, 4575.40it/s]

 87%|████████▋ | 358713/414113 [01:19<00:12, 4611.57it/s]

 87%|████████▋ | 359203/414113 [01:19<00:11, 4693.45it/s]

 87%|████████▋ | 359673/414113 [01:19<00:11, 4690.46it/s

 99%|█████████▉| 410407/414113 [01:31<00:00, 4613.25it/s]

 99%|█████████▉| 410870/414113 [01:31<00:00, 4590.39it/s]

 99%|█████████▉| 411330/414113 [01:31<00:00, 4580.03it/s]

 99%|█████████▉| 411799/414113 [01:31<00:00, 4609.95it/s]

100%|█████████▉| 412261/414113 [01:31<00:00, 4461.46it/s]

100%|█████████▉| 412736/414113 [01:31<00:00, 4543.59it/s]

100%|█████████▉| 413210/414113 [01:31<00:00, 4598.33it/s]

100%|█████████▉| 413674/414113 [01:31<00:00, 4610.38it/s]

100%|██████████| 414113/414113 [01:31<00:00, 4502.40it/s]Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth


  0%|          | 0/102502400 [00:00<?, ?it/s]

  1%|          | 524288/102502400 [00:00<00:21, 4672182.13it/s]

  3%|▎         | 3506176/102502400 [00:00<00:15, 6254395.00it/s]

  8%|▊         | 8052736/102502400 [00:00<00:11, 8436326.21it/s]

 15%|█▍        | 14950400/102502400 [00:00<00:07, 11451469.87it/s]

 23%|██▎       | 23961600/102502400 [00

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/3236], Loss: 3.5861, Perplexity: 36.09357
Epoch [1/3], Step [200/3236], Loss: 3.4049, Perplexity: 30.11127
Epoch [1/3], Step [300/3236], Loss: 3.2811, Perplexity: 26.6046
Epoch [1/3], Step [400/3236], Loss: 3.5139, Perplexity: 33.5790
Epoch [1/3], Step [500/3236], Loss: 2.9504, Perplexity: 19.1136
Epoch [1/3], Step [600/3236], Loss: 3.1138, Perplexity: 22.5056
Epoch [1/3], Step [700/3236], Loss: 3.1414, Perplexity: 23.1363
Epoch [1/3], Step [800/3236], Loss: 2.6448, Perplexity: 14.0810
Epoch [1/3], Step [900/3236], Loss: 2.7443, Perplexity: 15.5540
Epoch [1/3], Step [1000/3236], Loss: 2.5962, Perplexity: 13.4130
Epoch [1/3], Step [1100/3236], Loss: 2.6079, Perplexity: 13.5709
Epoch [1/3], Step [1200/3236], Loss: 2.5879, Perplexity: 13.3020
Epoch [1/3], Step [1300/3236], Loss: 2.3990, Perplexity: 11.0121
Epoch [1/3], Step [1400/3236], Loss: 2.4369, Perplexity: 11.4373
Epoch [1/3], Step [1500/3236], Loss: 2.5275, Perplexity: 12.52174
Epoch [1/3], Step [1600/3236], 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.